# Manual Fold Change Plotter

This notebook allows you to generate high-quality Fold Change plots by manually entering or pasting data from Excel.

### Recommended Workflow:
1.  **Prepare your table in Excel** with columns: `Condition`, `FoldChange`, and optionally `Primer`.
2.  **Copy the data** (including headers).
3.  **Choose an Input Method** below to load and plot your data.

In [ ]:
# SETUP: Run this cell first to initialize the plotting engine
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(color_codes=True, style="white")
pd.options.mode.chained_assignment = None

def plot_final_results(df, primer_order=None, condition_order=None, save_tag='ManualPlot', save_folder='./'):
    if df.empty: 
        print("DataFrame is empty!")
        return
    
    # Clean data: ensure numeric types
    df['FoldChange'] = pd.to_numeric(df['FoldChange'], errors='coerce')
    df = df.dropna(subset=['FoldChange'])
    
    x_col = 'Condition'
    hue_col = 'Primer' if 'Primer' in df.columns else None

    unique_x_values = df[x_col].unique()
    plot_order = condition_order if condition_order else list(unique_x_values)
    if condition_order:
        plot_order = [c for c in condition_order if c in unique_x_values]
        plot_order += [c for c in unique_x_values if c not in plot_order]

    width = max(5, len(plot_order) * 1.5)
    plt.figure(figsize=(width, 4))
    
    sns.barplot(
        data=df,
        x=x_col,
        y='FoldChange',
        hue=hue_col,
        hue_order=primer_order,
        order=plot_order,
        palette="tab10",
        errorbar=("ci", 95),
        capsize=0.5,
        alpha=0.8
    )

    sns.swarmplot(
        data=df,
        x=x_col,
        y='FoldChange',
        hue=hue_col,
        hue_order=primer_order,
        order=plot_order,
        dodge=True,
        color="black",
        alpha=0.6,
        legend=False
    )

    plt.ylabel("Fold Change", fontsize=14)
    plt.xlabel("")
    plt.xticks(rotation=30, ha='right')
    
    if hue_col:
        plt.legend(title="Primer", bbox_to_anchor=(1.03, 1), loc='upper left', frameon=False)
    
    plt.gca().spines[:].set_linewidth(1)
    plt.gca().tick_params(axis="both", which="major", labelsize=12, direction="in", bottom=True, left=True, length=5, width=1)

    if save_folder:
        save_path = os.path.join(save_folder, f"{save_tag}_FoldChange.png")
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to: {save_path}")
    plt.show()

## Method 1: Interactive Data Entry (Recommended)
*Note: This requires `ipywidgets`. If the box doesn't appear, run `!pip install ipywidgets` and restart the kernel.*

In [ ]:
try:
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    text_area = widgets.Textarea(
        value='',
        placeholder='Paste Excel data here (with headers)...',
        description='Paste Data:',
        layout={'width': '100%', 'height': '200px'}
    )
    save_tag_input = widgets.Text(value='MyManualPlot', description='File Name:')
    button = widgets.Button(description="Generate Plot", button_style='primary')
    output = widgets.Output()

    def on_button_clicked(b):
        with output:
            clear_output()
            raw_data = text_area.value.strip()
            if not raw_data: return
            try:
                df = pd.read_csv(io.StringIO(raw_data), sep='\t')
                df.columns = [c.strip() for c in df.columns]
                display(df.head())
                plot_final_results(df, save_tag=save_tag_input.value)
            except Exception as e:
                print(f"Error: {e}")

    button.on_click(on_button_clicked)
    display(widgets.VBox([text_area, save_tag_input, button]))
    display(output)
except ModuleNotFoundError:
    print("ipywidgets not found. Please use Method 2 below or install it using: !pip install ipywidgets")

## Method 2: Manual Data Block
If you don't want to use widgets, just paste your data into the triple quotes below and run the cell.

In [ ]:
PASTE_YOUR_DATA_HERE = """
Condition	FoldChange	Primer
DMSO	1.0	Target1
Drug A	0.5	Target1
Drug B	2.3	Target1
"""

try:
    df_manual = pd.read_csv(io.StringIO(PASTE_YOUR_DATA_HERE.strip()), sep='\t')
    plot_final_results(df_manual, save_tag="ManualBlockPlot")
except Exception as e:
    print(f"Error parsing data: {e}")

## Method 3: Direct Clipboard Input
*Note: This works only when running locally on your computer with a GUI.*

In [ ]:
# Copy in Excel, then run this cell
try:
    df_clipboard = pd.read_clipboard()
    display(df_clipboard.head())
    plot_final_results(df_clipboard, save_tag="ClipboardPlot")
except Exception as e:
    print(f"Clipboard access failed: {e}")